# Lesson 6.6.1 关于多分类评估指标的macro与micro过程

在正式讨论关于网格搜索的进阶使用方法之前，我们需要先补充一些关于多分类问题的评估指标计算过程。在此前的课程中，我们曾经介绍过分类模型在解决多分类问题时的不同策路，同时也介绍过二分类问题的更高级评估指标，如F1-score和ROC-AUC等，接下来我们将详细讨论关于多分类预测结果在F1-socre和ROC-AUC中的评估过程，以及在sklearn中如何调用函数进行计算。

In [1]:
# 科学计算模块
import numpy as np
import pandas as pd

# 画图模块
import matplotlib.pyplot as plt

# 机器学习模块
from ML_basic_function import *

# Scikit-Learn
# 评估模器模块
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import  LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import  GridSearchCV

# 实用函数
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 数据准备
from sklearn.datasets import load_iris


* 多分类F1-Score评估指标

首先导入和F1-Score相关的评估指标计算函数

In [2]:
from sklearn.metrics import  precision_score, recall_score, f1_score

然后简单查看相关说明文档，发现这几组合混淆矩阵相关的评估指标基本是共用了一套参数命名，并且大多数参数其实都是作用于多分类问题，对于二分类问题，我们可以简单调用相关函数直接计算：

In [3]:
y_true = np.array([1, 0, 0, 1, 0, 1])
y_pred = np.array([1, 1, 0, 1, 0, 1])

In [4]:
precision_score(y_true, y_pred), recall_score(y_true, y_pred), f1_score(y_true, y_pred)

(0.75, 1.0, 0.8571428571428571)

In [5]:
precision_score?

Signature:
precision_score(
    y_true,
    y_pred,
    *,
    labels=None,
    pos_label=1,
    average='binary',
    sample_weight=None,
    zero_division='warn',
)
Docstring:
Compute the precision.

The precision is the ratio ``tp / (tp + fp)`` where ``tp`` is the number of
true positives and ``fp`` the number of false positives. The precision is
intuitively the ability of the classifier not to label as positive a sample
that is negative.

The best value is 1 and the worst value is 0.

Support beyond term:`binary` targets is achieved by treating :term:`multiclass`
and :term:`multilabel` data as a collection of binary problems, one for each
label. For the :term:`binary` case, setting `average='binary'` will return
precision for `pos_label`. If `average` is not `'binary'`, `pos_label` is ignored
and precision for both classes are computed, then averaged or both returned (when
`average=None`). Similarly, for :term:`multiclass` and :term:`multilabel` targets,
precision for all `labels` 

![Alt text](image-118.png)

其中，需要重点介绍多分类问题时average参数不同取值时的计算方法。此处以recall为例进行计算，重点介绍当averagel取值为'macro'、'micro和'weighted的情况，其他指标也类似，例如有简单多分类问题如下：

![Alt text](image-119.png)

我们令1类标签为0；2类标签为1；3类标签为2，则上述数据集真实标签为：

In [6]:
y_true = np.array([0, 1, 2, 2, 0, 1, 1,2, 0, 2])

并且最终分类预测结果为：

In [7]:
y_pred = np.array([0, 1, 0, 2, 2, 1, 2, 2, 0, 2])

据此，我们可以构造多分类混淆矩阵如下：

![Alt text](image-120.png)

据此我们可以计算三个类别中的TP和FN

In [8]:
tp1 =2
tp2 = 2
tp3 = 3

In [9]:
fn1 = 1
fn2 = 1
fn3 = 1

接下来有两种计算recall的方法，其一是先计算每个类别的recall，然后求均值：

In [10]:
re1 = tp1/(tp1+fn1)
re2 = tp2/(tp2+fn2)
re3 = tp3/(tp3+fn3)

In [11]:
np.mean([re1, re2, re3])

0.6944444444444443

这也就是average参数取值为macro时的计算结果：

In [13]:
recall_score(y_true, y_pred, average='macro')

0.6944444444444443

当然，如果上述手动实现过程不求均值，而是根据每个类别的数量进行加权求和，则就是参数average参数取值为weighted时的结果：

In [14]:
recall_score(y_true, y_pred, average='weighted')

0.7

当然，还有另外一种计算方法，那就是先计算整体的TP和FN,然后根据整体TP和FN计算recall::

In [15]:
tp = tp1 + tp2 + tp3
fn = fn1 + fn2 + fn3
tp / (tp + fn)

0.7

In [16]:
recall_score(y_true, y_pred, average='micro')

0.7

对于上述三个不同参数的选取，首先如果是样本不平衡问题（如果是要侧重训练模型判别小类样本的能力的情况下）、则应排除weighted参数，以避免赋予大类样本更高的权重。除此以外，在大多数情况下这三个不同的参数其实并不会对最后评估器的选取结果造成太大影响，只是在很多要求严谨的场合下需要说明多分类的评估结果的计算过程，此时需要简单标注下是按照何种方法进行的计算。

不过，如果是混淆矩阵中相关指标和ROC-AUC指标放在一起讨论，由于新版sklearn中ROC-AUC本身不支持在多分类时按照micro算、只支持macro计算，因此建议混淆矩阵的多分类计算过程也选择macro过程，以保持一致。后续在没有进行其他特殊说明的情况下，课上统一采用macro指标进行多分类问题评估指标的计算。

> 不过值得注意的是，还有一种观点，尽管micro和macro方法在混淆矩阵相关指标的计算过程中差别不大，在roc
auc中，macro指标并不利于非平衡样本的计算(混淆矩阵中可以通过positive的类别选择来解决这一问题)，需要
配合ovr分类方法才能够有所改善。

* 多分类ROC-AUC评估指标

接下来继续讨论关于多分类的ROC-AUC评估指标的相关问题：

In [17]:
from sklearn.metrics import  roc_auc_score

能够发现，roc_auc score评估指标函数中大多数参数都和此前介绍的混淆矩阵中评估指标类似。接下来我们简单尝试使用roc-auc函数进行评估指标计算，根据roc-auc的计算流程可知，此处我们需要在y_prd参数位中输入模型概率预测结果：

In [18]:
y_true = np.array([1, 0, 0, 1, 0, 1])
y_pred = np.array([0.9, 0.7, 0.2, 0.7, 0.4, 0.8])   # 预测概率

In [19]:
roc_auc_score(y_true, y_pred)

0.9444444444444444

当然，如果我们在y_pred参数中输入分类结果，该函数也能计算出最终结果：

In [20]:
y_true = np.array([1, 0, 0, 1, 0, 1])
y_pred = np.array([1, 1, 0, 1, 0, 1])

In [21]:
roc_auc_score(y_true, y_pred)

0.8333333333333334

不过，此时模型会默认预测标签为0的概率结果为0.4、预测标签为1的概率预测结果为0.6，即上述结果等价于：

In [22]:
y_true = np.array([1, 0, 0, 1, 0, 1])
y_pred = np.array([0.6, 0.6, 0.4, 0.6, 0.4, 0.6])

In [23]:
roc_auc_score(y_true, y_pred)

0.8333333333333334

In [24]:
roc_auc_score?

Signature:
roc_auc_score(
    y_true,
    y_score,
    *,
    average='macro',
    sample_weight=None,
    max_fpr=None,
    multi_class='raise',
    labels=None,
)
Docstring:
Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC)     from prediction scores.

Note: this implementation can be used with binary, multiclass and
multilabel classification, but some restrictions apply (see Parameters).

Read more in the :ref:`User Guide <roc_metrics>`.

Parameters
----------
y_true : array-like of shape (n_samples,) or (n_samples, n_classes)
    True labels or binary label indicators. The binary and multiclass cases
    expect labels with shape (n_samples,) while the multilabel case expects
    binary label indicators with shape (n_samples, n_classes).

y_score : array-like of shape (n_samples,) or (n_samples, n_classes)
    Target scores.

    * In the binary case, it corresponds to an array of shape
      `(n_samples,)`. Both probability estimates and non-thresholded
     

![Alt text](image-121.png)

此处需要注意的是关于multi_class参数的选择。一般来说sklearn中的multi_class参数都是二分类器中用于解决多元分类问题时的参数（如逻辑回归），而由于roc-auc需要分类结果中的概率来完成最终计算，因此需要知道概率结果对应分类标签一即到底是以ovo还是ovr模式在进行多分类，因此如果是进行多分类roc-auc计算时，需要对其进行明确说明。

不过对于多分类逻辑回归来说，无论是ovr还是mvm策略，最终分类结果其实都可以看成是ovr分类结果，因此如果是多分类逻辑回归计算roc-auc,需要设置multi_class参数为ovr。同时由于根据roc-auc的函数参数说明可知，在muti_class参数取为ovr时，average参数取值为macro时能够保持一个较高的偏态样本敏感性，因此对于roc-auc来说，大多数时候average参数建议取值为macro。
总结一下，对于roc-auc进行多分类问题评估时，建议选择的参数组合是ovr/ovo+macro,而ovr/ovo的参数选择需要根据具体的多分类模型来定，如果是围绕逻辑回归多分类评估器来进行结果评估，则建议roc-auc和逻辑回归评估器的multi_class参数都选择ovr。